In [ ]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
import numpy as np
import copy

In [ ]:
ds = xr.open_dataset('../data/ecefiles/runoff_depth.nc')
ds

In [ ]:
proj = ccrs.SouthPolarStereo()
cmap = copy.copy(mpl.cm.get_cmap("cmo.thermal_r"));
cmap.set_under('w')
cmap.set_bad('w')
cmap.set_over('w')

In [ ]:
lon = ds.nav_lon
lat = ds.nav_lat
dep = ds['rodepth']
depp = dep.values[0,:,:]

In [ ]:
fig = plt.figure(figsize=(13,10))
ax = fig.add_subplot(111,projection=proj)
ax.set_extent([-180,180, -80, -53], crs=ccrs.PlateCarree())
im = ax.scatter(lon[:,1:-1],lat[:,1:-1],10,c=depp[:,1:-1],cmap=cmap,transform=ccrs.PlateCarree(),vmin=0,vmax=100)
ax.coastlines(lw=.1)
plt.colorbar(im)
plt.show()

In [ ]:
#Get distance from Antarctic coast
dss = xr.open_dataset('../data/ecefiles/masks.nc')
msk = dss['O1t0.msk']
dss.close()

mask = np.where(msk==1,1,1e12)[:,1:-1]
cdist = np.zeros((depp.shape))+1000
Re = 6371 #km

for j in range(depp.shape[0]):
    for i in range(depp.shape[1]):
        if lat[j,i] > -50:
            continue
        
        #Haversine formula
        dist = 2*Re*np.arcsin(np.sqrt(np.sin(np.deg2rad(0.5*(lat[:,1:-1]-lat[j,i])))**2+np.cos(np.deg2rad(lat[j,i]))*np.cos(np.deg2rad(lat[:,1:-1]))*np.sin(np.deg2rad(0.5*(lon[:,1:-1]-lon[j,i])))**2))
        dist = np.where(dist==0,1,dist)
        dist = np.where(lat[:,1:-1]>-60,1e12,dist)
        cdist[j,i] = np.minimum(1000,np.min(dist*mask))

In [ ]:
proj = ccrs.SouthPolarStereo()
fig = fig = plt.figure(figsize=(13,10))

ax = fig.add_subplot(111,projection=proj)
ax.set_extent([-180,180, -80, -53], crs=ccrs.PlateCarree())
im = ax.scatter(lon,lat,15,c=cdist,cmap=cmap,transform=ccrs.PlateCarree(),vmin=2,vmax=999)
ax.coastlines(lw=.1)

plt.colorbar(im)
plt.show()

In [ ]:
depp2 = np.where(np.logical_and(cdist>1,cdist<150),200,depp)

fig = fig = plt.figure(figsize=(24,8))

ax = fig.add_subplot(121,projection=proj)
ax.set_extent([-180,180, -80, -3], crs=ccrs.PlateCarree())
im = ax.scatter(lon,lat,15,c=depp,cmap=cmap,transform=ccrs.PlateCarree(),vmin=0,vmax=200)
ax.coastlines(lw=.1)

ax = fig.add_subplot(122,projection=proj)
ax.set_extent([-180,180, -80, -3], crs=ccrs.PlateCarree())
im = ax.scatter(lon,lat,15,c=depp2,cmap=cmap,transform=ccrs.PlateCarree(),vmin=0,vmax=200)
ax.coastlines(lw=.1)


plt.colorbar(im)
plt.show()

In [ ]:
#Save new depth file
ds['rodepth'][0,:,:] = depp2

ds.to_netcdf('../data/inputfiles/runoff_depth.nc')